In [2]:
import requests 
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import time
import re

In [ ]:
def donga_crawling(start_date, end_date):
    start_date = '-'.join(start_date.split('.'))
    end_date = '-'.join(end_date.split('.'))
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
    eng_url_head = 'https://www.donga.com/news/search?&query=kaist&check_news=1&more=1&sorting=2&search_date=1&v1=&v2=&range=3&p='
    kor_url_head = 'https://www.donga.com/news/search?&query=카이스트&check_news=1&more=1&sorting=2&search_date=1&v1=&v2=&range=3&p='
    # 검색 결과에 따른 url 링크 저장
    url1 = []
    # url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
    page = 1
    while True:
        url_final = eng_url_head + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            elements = soup.select('.searchCont a[href]')
            if len(elements) == 0:
                break
             # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            if 'pdf_viewer' in link:
                continue
            url1.append(link)
        page += 15
            
    # 검색 결과에 따른 url2 링크 저장
    url2 = []
    page = 1
    while True:
        url_final = kor_url_head + str(page)
        try:
            response = requests.get(url_final, headers=headers)
            html = response.content
            soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
            elements = soup.select('.searchCont a[href]')
            if len(elements) == 0:
                break
             # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
        except:
            print('Request Failed_1')
        for element in elements:
            link = element.attrs['href']
            if 'pdf_viewer' in link:
                continue
            url1.append(link)
        page += 15
        
    url = list(set(url1).union(set(url2)))
    print(len(url))
    if len(url)==0:
        print("Articles Not Found")
        return 0
    df = pd.DataFrame({"url": url}, columns=["url"])
    contents = []
    titles = []
    urls = []
    dates = []
    for row in tqdm(df.itertuples()):
        link = getattr(row, 'url')
        response = requests.get(link, headers=headers)
        html = response.text
        soup = bs(html,'html.parser')
        for script in soup(["script", "style"]):
                script.decompose()
        try:
            date = soup.select('.date01')[0].get_text()
            date = date.split(' ')[1]
            if start_date <= date and date <= end_date:
                title = soup.select('title')[0].get_text()
                title = title.replace('\xa0', " ")
                title = title.replace('\n', " ")
                content = soup.select('.article_txt')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
                content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
                titles.append(title)
                contents.append(content)
                urls.append(link)
        except:
            print(row)
    df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
    df.to_excel("articles/donga.xlsx",index = False)
    df.to_pickle("pickle files/donga.pkl")

In [12]:
donga_crawling('2015.01.01', '2020.10.23')

0it [00:00, ?it/s]

911


911it [05:35,  2.72it/s]


In [3]:
start_date = '-'.join('2015.01.01'.split('.'))
end_date = '-'.join('2020.10.23'.split('.'))
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
eng_url_head = 'https://www.donga.com/news/search?&query=kaist&check_news=1&more=1&sorting=2&search_date=1&v1=&v2=&range=3&p='
kor_url_head = 'https://www.donga.com/news/search?&query=카이스트&check_news=1&more=1&sorting=2&search_date=1&v1=&v2=&range=3&p='
# 검색 결과에 따른 url 링크 저장
url1 = []
# url 변화하는거 적당히 보고 알아서 잘 해야됨. 페이지 단위로 넘어가는게 아니라면 selenium 써야할 수도 있음.
page = 1
while True:
    url_final = eng_url_head + str(page)
    try:
        response = requests.get(url_final, headers=headers)
        html = response.content
        soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
        elements = soup.select('.searchCont a[href]')
        if len(elements) == 0:
            break
         # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
    except:
        print('Request Failed_1')
    for element in elements:
        link = element.attrs['href']
        if 'pdf_viewer' in link:
            continue
        url1.append(link)
    page += 15

In [6]:
url2 = []
page = 1
while True:
    url_final = kor_url_head + str(page)
    try:
        response = requests.get(url_final, headers=headers)
        html = response.content
        soup = bs(html, "html.parser")
#            page가 넘어갔을 때 검색결과가 없는 경우 while 문 break
        elements = soup.select('.searchCont a[href]')
        if len(elements) == 0:
            break
         # F12 눌러서 적당히 우리가 원하는 a href 태그 url 링크 달려있는 id 또는 class 찾아서 넣어주면 됨.
    except:
        print('Request Failed_1')
    for element in elements:
        link = element.attrs['href']
        if 'pdf_viewer' in link:
            continue
        url1.append(link)
    page += 15

url = list(set(url1).union(set(url2)))
print(len(url))
if len(url)==0:
    print("Articles Not Found")
df = pd.DataFrame({"url": url}, columns=["url"])

3329


In [9]:
len(df)

3329

In [10]:
contents = []
titles = []
urls = []
dates = []
for row in tqdm(df.itertuples()):
    link = getattr(row, 'url')
    response = requests.get(link, headers=headers)
    html = response.text
    soup = bs(html,'html.parser')
    for script in soup(["script", "style"]):
            script.decompose()
    try:
        date = soup.select('.date01')[0].get_text()
        date = date.split(' ')[1]
        if start_date <= date and date <= end_date:
            title = soup.select('title')[0].get_text()
            title = title.replace('\xa0', " ")
            title = title.replace('\n', " ")
            content = soup.select('.article_txt')[0].get_text() # select 안에 본문에 해당하는 id 값을 찾아서 넣어주면 됨.
            content = content.replace('\xa0', " ").replace('\u3000','').replace('\r','').replace('\n', " ")
            titles.append(title)
            contents.append(content)
            urls.append(link)
    except:
        print(row)

df = pd.DataFrame({"url": urls, "title": titles, "content": contents}, columns=["url", "title", "content"])
df.to_excel("articles/donga.xlsx",index = False)
df.to_pickle("pickle files/donga.pkl")

3329it [22:32,  2.46it/s]
